## Import the libraries

In [1]:
!pip3 install dataframe_image
!pip3 install pycaret
!pip3 install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -U scikit-learn pandas numpy matplotlib "pycaret==3.3.2"


  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.7-cp310-cp310-win_amd64.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import resample
from pycaret.classification import *
#import dataframe_image as dfi
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import os
from pathlib import Path


In [4]:
!python --version


Python 3.10.11


##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Parameter List.

#### Setup the model creation with pycaret for Long Parameter List detection

In [5]:
df = pd.read_csv('dataset_long_parameter_list.csv')
print(f"Dataset carregado com {df.shape[0]} instâncias e {df.shape[1]-1} features")
print(f"\n📊 Distribuição das classes:")
class_counts = df['smell_label'].value_counts()
print(class_counts)

# Calcular proporção de desbalanceamento
ratio = class_counts.iloc[0] / class_counts.iloc[1]
print(f"\n⚠️  Proporção de desbalanceamento: {ratio:.1f}:1")
print(f"   Classe minoritária representa: {(class_counts.iloc[1]/len(df)*100):.2f}% do dataset")

if ratio > 10:
    print("🚨 ALERTA: Desbalanceamento extremo detectado!")

print(f"\nPrimeiras 5 linhas:")
df.head()

Dataset carregado com 10696 instâncias e 13 features

📊 Distribuição das classes:
smell_label
non-long-parameter-list    10480
long-parameter-list          216
Name: count, dtype: int64

⚠️  Proporção de desbalanceamento: 48.5:1
   Classe minoritária representa: 2.02% do dataset
🚨 ALERTA: Desbalanceamento extremo detectado!

Primeiras 5 linhas:


,raw_sloc,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,smell_label
0,9,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
1,2,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
2,3,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
3,3,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
4,29,0,0,0,13,17,21,54.62919,85.83672,2.708333,232.474449,12.915247,0.028612,non-long-parameter-list


In [6]:
# Balanceamento MANUAL se SMOTE automático falhar
from sklearn.utils import resample

print("🔧 Analisando necessidade de balanceamento manual...")

# Verificar distribuição atual
class_counts = df['smell_label'].value_counts()
print(f'Distribuição original:')
print(class_counts)

# Calcular proporção
ratio = class_counts.iloc[0] / class_counts.iloc[1]
print(f'\nProporção de desbalanceamento: {ratio:.1f}:1')

# Se proporção > 20:1, fazer balanceamento manual mais agressivo
if ratio > 20:
    print('\n⚠️  Desbalanceamento extremo detectado!')
    print('🔧 Aplicando balanceamento manual agressivo...')
    
    # Separar classes
    df_majority = df[df['smell_label'] == 'non-long-parameter-list']
    df_minority = df[df['smell_label'] == 'long-parameter-list']
    
    # Estratégia agressiva: aumentar minoritária para 15% da majoritária
    target_minority_size = int(len(df_majority) * 0.15)
    print(f'   Alvo: {len(df_majority)}:{target_minority_size} ({len(df_majority)/target_minority_size:.1f}:1)')
    
    # Fazer oversampling da classe minoritária
    df_minority_upsampled = resample(
        df_minority, 
        replace=True, 
        n_samples=target_minority_size,
        random_state=42
    )
    
    # Combinar
    df_balanced = pd.concat([df_majority, df_minority_upsampled])
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f'✅ Balanceamento manual concluído:')
    print(f'   Novo shape: {df_balanced.shape}')
    print(f'   Nova distribuição:')
    print(df_balanced['smell_label'].value_counts())
    
    new_ratio = len(df_majority) / target_minority_size
    print(f'   Nova proporção: {new_ratio:.1f}:1')
    
    # Usar DataFrame balanceado
    df = df_balanced
else:
    print('✅ Desbalanceamento dentro dos limites aceitáveis')

print('\n📊 Dataset pronto para o setup!')
df.head()

🔧 Analisando necessidade de balanceamento manual...
Distribuição original:
smell_label
non-long-parameter-list    10480
long-parameter-list          216
Name: count, dtype: int64

Proporção de desbalanceamento: 48.5:1

⚠️  Desbalanceamento extremo detectado!
🔧 Aplicando balanceamento manual agressivo...
   Alvo: 10480:1572 (6.7:1)
✅ Balanceamento manual concluído:
   Novo shape: (12052, 14)
   Nova distribuição:
smell_label
non-long-parameter-list    10480
long-parameter-list         1572
Name: count, dtype: int64
   Nova proporção: 6.7:1

📊 Dataset pronto para o setup!


,raw_sloc,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,smell_label
0,4,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,non-long-parameter-list
1,3,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,non-long-parameter-list
2,5,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,non-long-parameter-list
3,2,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,non-long-parameter-list
4,6,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,long-parameter-list


In [7]:
# Setup SIMPLIFICADO (com balanceamento manual já aplicado)
print("🔧 Configurando setup simplificado (balanceamento manual já aplicado)...")

s = setup(
    data=df,  # DataFrame já balanceado manualmente
    target='smell_label',
    session_id=42,
    fold=5,
    
    # Sem balanceamento automático (já foi feito manualmente)
    fix_imbalance=False,  # DESABILITADO - já feito manualmente
    
    # Preprocessing MÍNIMO para máximo desempenho
    normalize=True,
    transformation=False,
    feature_selection=False,
    remove_multicollinearity=False,
    pca=False,
    
)

print("✅ Setup configurado com sucesso!")
print("📋 Estratégia aplicada:")
print("   - Balanceamento manual prévio (15% sampling)")
print("   - Sem SMOTE automático (já aplicado)")
print("   - Preprocessing mínimo para máxima performance")
print("   - 5 folds para estabilidade")

🔧 Configurando setup simplificado (balanceamento manual já aplicado)...


,Description,Value
0,Session id,42
1,Target,smell_label
2,Target type,Binary
3,Target mapping,"long-parameter-list: 0, non-long-parameter-list: 1"
4,Original data shape,"(12052, 14)"
5,Transformed data shape,"(12052, 14)"
6,Transformed train set shape,"(8436, 14)"
7,Transformed test set shape,"(3616, 14)"
8,Numeric features,13
9,Preprocess,True


✅ Setup configurado com sucesso!
📋 Estratégia aplicada:
   - Balanceamento manual prévio (15% sampling)
   - Sem SMOTE automático (já aplicado)
   - Preprocessing mínimo para máxima performance
   - 5 folds para estabilidade


In [8]:
# from imblearn.over_sampling import SMOTE
# from pycaret.classification import setup

# s = setup(
#     data=df,
#     target='smell_label',
#     session_id=42,
#     fold=5,
#     fold_shuffle=True,
#     data_split_shuffle=True,
#     data_split_stratify=True,
#     train_size=0.8,                 # gera um hold-out interno
#     fix_imbalance=True,
#     fix_imbalance_method=SMOTE(k_neighbors=5, random_state=42),
#     normalize=True,
#     transformation=False,
#     feature_selection=False,
#     remove_multicollinearity=False,
#     pca=False,
#     verbose=False
# )

# s


In [9]:
# Verificar resultados do setup otimizado
df_processed = get_config('X')
print("📊 Análise após preprocessing otimizado:")
print(f"Shape do dataset processado: {df_processed.shape}")

# Verificar distribuição das classes depois do SMOTE
y_train = get_config('y_train')
print(f"\n📈 Distribuição das classes no treino (após SMOTE otimizado):")
new_class_counts = y_train.value_counts()
print(new_class_counts)

# Calcular nova proporção
new_ratio = new_class_counts.iloc[0] / new_class_counts.iloc[1]
print(f"\n✅ Nova proporção: {new_ratio:.1f}:1 (melhorou de {ratio:.1f}:1)")

# Verificar se o balanceamento melhorou
if new_ratio < ratio:
    improvement = ((ratio - new_ratio) / ratio) * 100
    print(f"🎯 Melhoria no balanceamento: {improvement:.1f}%")

print(f"\n📋 Estatísticas das features processadas:")
print(df_processed.describe().round(2))

# Verificar variância para garantir que não temos features constantes
print(f"\n🔍 Variância das features (valores baixos podem indicar problemas):")
variances = df_processed.var().sort_values()
print(variances.round(4))

# Alertar se houver variância muito baixa
low_var_features = variances[variances < 0.01]
if len(low_var_features) > 0:
    print(f"\n⚠️  Features com variância muito baixa: {list(low_var_features.index)}")
else:
    print("\n✅ Todas as features têm variância adequada")

📊 Análise após preprocessing otimizado:
Shape do dataset processado: (12052, 13)

📈 Distribuição das classes no treino (após SMOTE otimizado):
smell_label
non-long-parameter-list    7336
long-parameter-list        1100
Name: count, dtype: int64

✅ Nova proporção: 6.7:1 (melhorou de 48.5:1)
🎯 Melhoria no balanceamento: 86.3%

📋 Estatísticas das features processadas:
       raw_sloc  raw_multi  raw_blank  raw_single_comments  hal_func_N2  \
count  12052.00   12052.00   12052.00             12052.00     12052.00   
mean       8.82       3.38       0.77                 0.05         6.92   
std       11.40      13.32       3.55                 0.40        20.33   
min        1.00       0.00       0.00                 0.00         0.00   
25%        3.00       0.00       0.00                 0.00         0.00   
50%        6.00       0.00       0.00                 0.00         1.00   
75%       10.00       0.00       0.00                 0.00         6.00   
max      196.00     263.00     1

# Comparar modelos com setup otimizado para desbalanceamento extremo
print("🚀 Comparando modelos com setup otimizado...")
print("Estratégia: Foco em F1-score para classe minoritária")

top5_models = compare_models(
    n_select=13, 
    sort='f1',  # Otimizar para F1-score
    turbo=True  # Acelerar comparação
)

In [10]:
top5_models = compare_models(n_select=13, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9462,0.9426,0.9462,0.9442,0.9431,0.7370,0.7455,0.3460
knn,K Neighbors Classifier,0.9087,0.8974,0.9087,0.9126,0.9103,0.6118,0.6131,0.3580
lda,Linear Discriminant Analysis,0.8670,0.7526,0.8670,0.8323,0.8374,0.1855,0.2195,0.3920
qda,Quadratic Discriminant Analysis,0.8276,0.7283,0.8276,0.8244,0.8258,0.2247,0.2252,0.0140
lr,Logistic Regression,0.8706,0.7523,0.8706,0.8349,0.8253,0.0981,0.1623,0.4020
ridge,Ridge Classifier,0.8716,0.7519,0.8716,0.8433,0.8229,0.0816,0.1575,0.0100
nb,Naive Bayes,0.8249,0.7274,0.8249,0.8154,0.8196,0.1833,0.1846,0.3320
svm,SVM - Linear Kernel,0.8592,0.6173,0.8592,0.8062,0.8178,0.0762,0.0923,0.0140
dummy,Dummy Classifier,0.8696,0.5000,0.8696,0.7562,0.8090,0.0000,0.0000,0.0140


In [11]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongParameterList_results.latex',index=False)
df3.to_csv('LongParameterList_results.csv',index=False)
#dfi.export(df2, 'Bin_Blob_FS1.png')

In [12]:
from sklearn.metrics import classification_report, precision_recall_curve
import numpy as np

print("🎯 Análise detalhada com Threshold Tuning para classes desbalanceadas")
print("="*80)

# Dicionário para armazenar os relatórios e melhores thresholds
reports = {}
best_thresholds = {}
data = []

# Iterar sobre os modelos
for i, model in enumerate(top5_models):
    model_name = model.__class__.__name__
    print(f"\n📊 {i+1}. Analisando modelo: {model_name}")
    
    # Gerar predições com probabilidades
    try:
        predictions = predict_model(model, raw_score=True)
        has_scores = True
    except:
        predictions = predict_model(model)
        has_scores = False
    
    # Obter scores para threshold tuning - TRATAMENTO PARA DIFERENTES TIPOS DE MODELOS
    y_scores = None
    
    if has_scores and 'prediction_score' in predictions.columns:
        # PyCaret forneceu scores diretamente
        y_scores = predictions['prediction_score']
        print(f"   ✅ Usando scores do PyCaret")
    else:
        # Tentar obter probabilidades do modelo (se suportado)
        try:
            # Verificar se o modelo tem predict_proba
            if hasattr(model, 'predict_proba'):
                X_test = get_config('X_test')
                y_proba = model.predict_proba(X_test)[:, 1]
                y_scores = y_proba[:len(predictions)]
                print(f"   ✅ Usando predict_proba do modelo")
            else:
                # Modelos que não suportam predict_proba (Ridge, LinearDiscriminant, etc.)
                # Usar decision function se disponível
                if hasattr(model, 'decision_function'):
                    X_test = get_config('X_test')
                    y_decision = model.decision_function(X_test)
                    # Para classificação binária, decision_function retorna array 1D
                    y_scores = y_decision[:len(predictions)]
                    print(f"   ✅ Usando decision_function")
                else:
                    # Último recurso: usar predições binárias como scores
                    y_scores = (predictions['prediction_label'] == 'long-parameter-list').astype(float)
                    print(f"   ⚠️  Usando predições binárias (sem probabilidades)")
        except Exception as e:
            print(f"   ⚠️  Erro ao obter scores: {str(e)}")
            # Fallback: usar predições binárias
            y_scores = (predictions['prediction_label'] == 'long-parameter-list').astype(float)
    
    # Preparar dados para análise
    y_true = predictions['smell_label'] == 'long-parameter-list'
    y_pred_default = predictions['prediction_label'] == 'long-parameter-list'
    
    # Se não conseguirmos scores variados, pular threshold tuning
    if len(np.unique(y_scores)) <= 1:
        print(f"   ⚠️  Scores constantes, pulando threshold tuning")
        best_threshold = 0.5
        report_optimized = classification_report(
            y_true, 
            y_pred_default,
            target_names=['non-long-parameter-list', 'long-parameter-list'],
            output_dict=True
        )
        opt_f1 = report_optimized['long-parameter-list']['f1-score']
        def_f1 = opt_f1
        improvement = 0.0
    else:
        # Encontrar melhor threshold usando precision-recall curve
        try:
            precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            best_idx = np.argmax(f1_scores)
            best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
            best_f1 = f1_scores[best_idx]
            
            # Predições com melhor threshold
            y_pred_optimized = y_scores >= best_threshold
            
            # Gerar relatório com threshold otimizado
            report_optimized = classification_report(
                y_true, 
                y_pred_optimized,
                target_names=['non-long-parameter-list', 'long-parameter-list'],
                output_dict=True
            )
            
            # Gerar relatório com threshold padrão (0.5)
            report_default = classification_report(
                y_true, 
                y_pred_default,
                target_names=['non-long-parameter-list', 'long-parameter-list'],
                output_dict=True
            )
            
            # Comparar desempenho
            opt_f1 = report_optimized['long-parameter-list']['f1-score']
            def_f1 = report_default['long-parameter-list']['f1-score']
            improvement = ((opt_f1 - def_f1) / def_f1 * 100) if def_f1 > 0 else 0
            
            print(f"   Threshold padrão (0.5): F1 = {def_f1:.3f}")
            print(f"   Threshold otimizado ({best_threshold:.3f}): F1 = {opt_f1:.3f}")
            print(f"   Melhoria: {improvement:+.1f}%")
            
        except Exception as e:
            print(f"   ⚠️  Erro no threshold tuning: {str(e)}")
            best_threshold = 0.5
            report_optimized = classification_report(
                y_true, 
                y_pred_default,
                target_names=['non-long-parameter-list', 'long-parameter-list'],
                output_dict=True
            )
            opt_f1 = report_optimized['long-parameter-list']['f1-score']
            def_f1 = opt_f1
            improvement = 0.0
    
    # Armazenar informações
    reports[model_name] = {
        'optimized': report_optimized,
        'default': report_default if 'report_default' in locals() else report_optimized
    }
    best_thresholds[model_name] = best_threshold
    
    # Armazenar para DataFrame
    data.append({
        "Model": model_name,
        "Rank": i + 1,
        "Best_Threshold": best_threshold,
        "F1_Optimized": opt_f1,
        "F1_Default": def_f1,
        "Improvement_%": improvement,
        "Precision_Opt": report_optimized['long-parameter-list']['precision'],
        "Recall_Opt": report_optimized['long-parameter-list']['recall'],
        "Accuracy": report_optimized['accuracy']
    })

# Criar DataFrame com resultados
results_df = pd.DataFrame(data)

# Ordenar por F1 otimizado
results_df = results_df.sort_values('F1_Optimized', ascending=False)

print("\n" + "="*80)
print("🏆 RANKING DOS MODELOS (COM THRESHOLD OTIMIZADO)")
print("="*80)
print(results_df[['Model', 'Rank', 'F1_Optimized', 'F1_Default', 'Best_Threshold', 'Improvement_%']].round(3).to_string(index=False))

print(f"\n📈 Melhoria média com threshold tuning: {results_df['Improvement_%'].mean():+.1f}%")
print(f"🎯 Melhor modelo: {results_df.iloc[0]['Model']} (F1: {results_df.iloc[0]['F1_Optimized']:.3f})")

🎯 Análise detalhada com Threshold Tuning para classes desbalanceadas

📊 1. Analisando modelo: LGBMClassifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9566,0.9530,0.9566,0.9553,0.9548,0.7933,0.7981


   ✅ Usando predict_proba do modelo
   Threshold padrão (0.5): F1 = 0.818
   Threshold otimizado (0.765): F1 = 0.311
   Melhoria: -62.0%

📊 2. Analisando modelo: KNeighborsClassifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9306,0.9225,0.9306,0.9324,0.9314,0.7014,0.7018


   ✅ Usando predict_proba do modelo
   Threshold padrão (0.5): F1 = 0.742
   Threshold otimizado (0.800): F1 = 0.270
   Melhoria: -63.6%

📊 3. Analisando modelo: LinearDiscriminantAnalysis


...

   ✅ Usando predict_proba do modelo
   Threshold padrão (0.5): F1 = 0.319
   Threshold otimizado (1.000): F1 = 0.275
   Melhoria: -13.8%

📊 8. Analisando modelo: SGDClassifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,SVM - Linear Kernel,0.8446,0,0.8446,0.7854,0.8085,0.0382,0.0457


   ✅ Usando decision_function
   Threshold padrão (0.5): F1 = 0.099
   Threshold otimizado (14.819): F1 = 0.372
   Melhoria: +274.6%

📊 9. Analisando modelo: DummyClassifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.8695,0.5000,0.8695,0.7560,0.8088,0.0000,0.0000


   ✅ Usando predict_proba do modelo
   ⚠️  Scores constantes, pulando threshold tuning

🏆 RANKING DOS MODELOS (COM THRESHOLD OTIMIZADO)
                        Model  Rank  F1_Optimized  F1_Default  Best_Threshold  Improvement_%
           LogisticRegression     5         0.381       0.149           1.000        156.454
                SGDClassifier     8         0.372       0.099          14.819        274.584
   LinearDiscriminantAnalysis     3         0.370       0.254           1.000         45.659
              RidgeClassifier     6         0.370       0.095          17.203        289.969
               LGBMClassifier     1         0.311       0.818           0.765        -61.976
QuadraticDiscriminantAnalysis     4         0.302       0.359           1.000        -15.699
                   GaussianNB     7         0.275       0.319           1.000        -13.824
         KNeighborsClassifier     2         0.270       0.742           0.800        -63.583
              DummyClassifi

In [13]:
# Salvar resultados detalhados
print("💾 Salvando resultados detalhados...")

# Salvar DataFrame completo com métricas
results_df.to_csv("longparameterlist_results_optimized.csv", index=False)
print("✅ Resultados completos salvos em 'longparameterlist_results_optimized.csv'")

# Salvar ranking simplificado
ranking_df = results_df[['Model', 'F1_Optimized', 'Best_Threshold', 'Improvement_%']].round(3)
ranking_df.to_csv("longparameterlist_ranking.csv", index=False)
print("✅ Ranking salvo em 'longparameterlist_ranking.csv'")

# Salvar resultados para LaTeX
results_df.round(3).to_latex('longparameterlist_results_optimized.latex', index=False)
print("✅ Resultados salvos em LaTeX para paper")

# Estatísticas adicionais
print(f"\n📊 ESTATÍSTICAS DA OTIMIZAÇÃO:")
print(f"   Modelos testados: {len(results_df)}")
print(f"   F1-score médio (otimizado): {results_df['F1_Optimized'].mean():.3f}")
print(f"   F1-score médio (padrão): {results_df['F1_Default'].mean():.3f}")
print(f"   Melhor F1 obtido: {results_df['F1_Optimized'].max():.3f}")
print(f"   Pior F1 obtido: {results_df['F1_Optimized'].min():.3f}")
print(f"   Desvio padrão: {results_df['F1_Optimized'].std():.3f}")

# Verificar se atingiu métricas mínimas aceitáveis
acceptable_f1 = 0.3  # 30% como mínimo aceitável
good_models = results_df[results_df['F1_Optimized'] >= acceptable_f1]

print(f"\n🎯 MODELOS ACEITÁVEIS (F1 ≥ {acceptable_f1}): {len(good_models)}/{len(results_df)}")
if len(good_models) > 0:
    print(good_models[['Model', 'F1_Optimized', 'Best_Threshold']].round(3).to_string(index=False))
else:
    print("⚠️  Nenhum modelo atingiu o F1-score mínimo aceitável.")
    print("   Considere:")
    print("   - Coletar mais dados da classe minoritária")
    print("   - Usar técnicas de ensemble especializadas")
    print("   - Revisar as features utilizadas")

💾 Salvando resultados detalhados...
✅ Resultados completos salvos em 'longparameterlist_results_optimized.csv'
✅ Ranking salvo em 'longparameterlist_ranking.csv'
✅ Resultados salvos em LaTeX para paper

📊 ESTATÍSTICAS DA OTIMIZAÇÃO:
   Modelos testados: 9
   F1-score médio (otimizado): 0.295
   F1-score médio (padrão): 0.315
   Melhor F1 obtido: 0.381
   Pior F1 obtido: 0.000
   Desvio padrão: 0.119

🎯 MODELOS ACEITÁVEIS (F1 ≥ 0.3): 6/9
                        Model  F1_Optimized  Best_Threshold
           LogisticRegression         0.381           1.000
                SGDClassifier         0.372          14.819
   LinearDiscriminantAnalysis         0.370           1.000
              RidgeClassifier         0.370          17.203
               LGBMClassifier         0.311           0.765
QuadraticDiscriminantAnalysis         0.302           1.000


In [14]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('longparameterlist_performance.latex',index=False)
df3.to_csv('longparameterlist_performance.csv',index=False)
print("Métricas de desempenho salvas em arquivos .latex e .csv")

Métricas de desempenho salvas em arquivos .latex e .csv


In [15]:
# Selecionar o melhor modelo baseado no F1 otimizado
best_model_name = results_df.iloc[0]['Model']
best_model = None

# Encontrar o objeto do melhor modelo
for model in top5_models:
    if model.__class__.__name__ == best_model_name:
        best_model = model
        break

if best_model is not None:
    print(f"🏆 MELHOR MODELO SELECIONADO: {best_model_name}")
    print(f"   F1-Score Otimizado: {results_df.iloc[0]['F1_Optimized']:.3f}")
    print(f"   Melhor Threshold: {results_df.iloc[0]['Best_Threshold']:.3f}")
    print(f"   Melhoria: {results_df.iloc[0]['Improvement_%']:+.1f}%")
    
    # Exibir hiperparâmetros do melhor modelo
    print(f"\n🔧 Hiperparâmetros:")
    print(best_model)
else:
    print("❌ Erro: Não foi possível encontrar o melhor modelo")
    best_model = top5_models[0]  # Fallback

🏆 MELHOR MODELO SELECIONADO: LogisticRegression
   F1-Score Otimizado: 0.381
   Melhor Threshold: 1.000
   Melhoria: +156.5%

🔧 Hiperparâmetros:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


#### Ensemble the top-5 models

In [16]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8821,0.0000,0.8821,0.8712,0.8436,0.1955,0.2906
1,0.8856,0.0000,0.8856,0.8795,0.8497,0.2289,0.3275
2,0.8779,0.0000,0.8779,0.8522,0.8462,0.2207,0.2773
3,0.8809,0.0000,0.8809,0.8594,0.8489,0.2332,0.2978
4,0.8832,0.0000,0.8832,0.8708,0.8473,0.2176,0.3059
Mean,0.8819,0.0000,0.8819,0.8666,0.8471,0.2192,0.2998
Std,0.0026,0.0000,0.0026,0.0096,0.0021,0.0131,0.0168


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [17]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9566,0.9530,0.9566,0.9553,0.9548,0.7933,0.7981


In [18]:
result = pull()

In [19]:
result.to_csv('LongParameterList_ensemble.csv')
print("Resultados do ensemble salvos em 'LongParameterList_ensemble.csv'")

Resultados do ensemble salvos em 'LongParameterList_ensemble.csv'


#### Plot the ensemble model

#### Finalize and save the ensemble model

In [20]:
# Finalizar e salvar o melhor modelo com threshold otimizado
candidate = best_model

print(f"💾 Finalizando o melhor modelo: {candidate.__class__.__name__}")

try:
    final_best = finalize_model(candidate)
    print("✅ Modelo finalizado com sucesso!")
except Exception as e:
    print(f"⚠️  finalize_model falhou: {str(e)}")
    print("🔧 Tentando workaround...")
    set_config('fix_imbalance', False)
    final_best = finalize_model(candidate)
    print("✅ Modelo finalizado com workaround!")

# Salvar o modelo final
model_filename = 'long_parameter_list_optimized'
save_model(final_best, model_filename)

print(f"✅ Modelo salvo como: {model_filename}.pkl")

# Salvar informações importantes para uso futuro
model_info = {
    'model_name': best_model_name,
    'best_threshold': results_df.iloc[0]['Best_Threshold'],
    'f1_score': results_df.iloc[0]['F1_Optimized'],
    'precision': results_df.iloc[0]['Precision_Opt'],
    'recall': results_df.iloc[0]['Recall_Opt'],
    'accuracy': results_df.iloc[0]['Accuracy'],
    'improvement_%': results_df.iloc[0]['Improvement_%']
}

# Salvar metadados do modelo
import json
with open(f"{model_filename}_metadata.json", 'w') as f:
    json.dump(model_info, f, indent=2)

print(f"✅ Metadados salvos em: {model_filename}_metadata.json")
print(f"\n📋 INFORMAÇÕES PARA USO FUTURO:")
print(f"   Modelo: {model_info['model_name']}")
print(f"   Threshold ótimo: {model_info['best_threshold']:.3f}")
print(f"   F1-Score esperado: {model_info['f1_score']:.3f}")
print(f"   Precision: {model_info['precision']:.3f}")
print(f"   Recall: {model_info['recall']:.3f}")

final_best

💾 Finalizando o melhor modelo: LogisticRegression
✅ Modelo finalizado com sucesso!
Transformation Pipeline and Model Successfully Saved
✅ Modelo salvo como: long_parameter_list_optimized.pkl
✅ Metadados salvos em: long_parameter_list_optimized_metadata.json

📋 INFORMAÇÕES PARA USO FUTURO:
   Modelo: LogisticRegression
   Threshold ótimo: 1.000
   F1-Score esperado: 0.381
   Precision: 0.282
   Recall: 0.587


Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['raw_sloc', 'raw_multi',
                                             'raw_blank', 'raw_single_comments',
                                             'hal_func_N2',
                                             'hal_func_vocabulary',
                                             'hal_func_length',
                                             'hal_func_calculated_length',...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                                                               with_std=True))),
                ('actual_estimator',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=42,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [21]:
save_model(final_best, 'long_parameter_list')
print("Modelo final salvo como 'long_parameter_list.pkl'")

Transformation Pipeline and Model Successfully Saved
Modelo final salvo como 'long_parameter_list.pkl'


#### Salvar os 5 melhores modelos individualmente

In [22]:
# Salvar os 5 melhores modelos com seus thresholds otimizados
print("💾 Salvando os 5 melhores modelos com thresholds otimizados...")

finalized_models = []
top_5_results = results_df.head(5)

for i, (_, row) in enumerate(top_5_results.iterrows()):
    model_name = row['Model']
    best_threshold = row['Best_Threshold']
    
    print(f"\n{i+1}. Processando {model_name}:")
    print(f"   Threshold ótimo: {best_threshold:.3f}")
    print(f"   F1-Score: {row['F1_Optimized']:.3f}")
    
    # Encontrar o modelo correspondente
    model_obj = None
    for model in top5_models:
        if model.__class__.__name__ == model_name:
            model_obj = model
            break
    
    if model_obj is not None:
        try:
            # Finalizar modelo
            final_model = finalize_model(model_obj)
            
            # Criar nome descritivo
            filename = f"{model_name.lower()}_lpl_opt_f1{row['F1_Optimized']:.2f}"
            
            # Salvar modelo
            save_model(final_model, filename)
            print(f"   ✅ Salvo como: {filename}.pkl")
            
            # Salvar metadados do modelo
            model_metadata = {
                'model_name': model_name,
                'best_threshold': best_threshold,
                'f1_score': row['F1_Optimized'],
                'precision': row['Precision_Opt'],
                'recall': row['Recall_Opt'],
                'accuracy': row['Accuracy'],
                'improvement_%': row['Improvement_%'],
                'rank': i + 1
            }
            
            metadata_file = f"{filename}_metadata.json"
            with open(metadata_file, 'w') as f:
                json.dump(model_metadata, f, indent=2)
            
            print(f"   ✅ Metadados salvos: {metadata_file}")
            
            finalized_models.append({
                'name': model_name,
                'filename': filename,
                'threshold': best_threshold,
                'f1_score': row['F1_Optimized'],
                'rank': i + 1
            })
            
        except Exception as e:
            print(f"   ❌ Erro ao salvar {model_name}: {str(e)}")
    else:
        print(f"   ❌ Modelo {model_name} não encontrado")

print(f"\n🎉 {len(finalized_models)} modelos salvos com sucesso!")
print("\n📊 RESUMO DOS MODELOS SALVOS:")
for model_info in finalized_models:
    print(f"   {model_info['rank']}. {model_info['name']}")
    print(f"      📁 {model_info['filename']}.pkl")
    print(f"      🎯 Threshold: {model_info['threshold']:.3f}")
    print(f"      📈 F1-Score: {model_info['f1_score']:.3f}")

# Salvar resumo geral
summary_file = 'long_parameter_list_models_summary.json'
with open(summary_file, 'w') as f:
    json.dump(finalized_models, f, indent=2)

print(f"\n✅ Resumo geral salvo em: {summary_file}")

💾 Salvando os 5 melhores modelos com thresholds otimizados...

1. Processando LogisticRegression:
   Threshold ótimo: 1.000
   F1-Score: 0.381
Transformation Pipeline and Model Successfully Saved
   ✅ Salvo como: logisticregression_lpl_opt_f10.38.pkl
   ✅ Metadados salvos: logisticregression_lpl_opt_f10.38_metadata.json

2. Processando SGDClassifier:
   Threshold ótimo: 14.819
   F1-Score: 0.372
Transformation Pipeline and Model Successfully Saved
   ✅ Salvo como: sgdclassifier_lpl_opt_f10.37.pkl
   ✅ Metadados salvos: sgdclassifier_lpl_opt_f10.37_metadata.json

3. Processando LinearDiscriminantAnalysis:
   Threshold ótimo: 1.000
   F1-Score: 0.370
Transformation Pipeline and Model Successfully Saved
   ✅ Salvo como: lineardiscriminantanalysis_lpl_opt_f10.37.pkl
   ✅ Metadados salvos: lineardiscriminantanalysis_lpl_opt_f10.37_metadata.json

4. Processando RidgeClassifier:
   Threshold ótimo: 17.203
   F1-Score: 0.370
Transformation Pipeline and Model Successfully Saved
   ✅ Salvo como:

In [23]:
# Salvar os 5 melhores modelos individualmente
print("Salvando os 5 melhores modelos para Long Parameter List detection...")

# Lista para armazenar os modelos finalizados e seus nomes
finalized_models = []

# Iterar sobre os 5 melhores modelos
for i, model in enumerate(top5_models[:5]):
    # Obter nome do modelo
    model_name = model.__class__.__name__
    print(f"\n{i+1}. Processando modelo: {model_name}")
    
    # Finalizar o modelo
    try:
        final_model = finalize_model(model)
        
        # Criar nome descritivo para o arquivo
        filename = f"{model_name.lower()}_long_parameter_list"
        
        # Salvar o modelo
        save_model(final_model, filename)
        print(f"   Modelo salvo como: {filename}.pkl")
        
        # Adicionar à lista de modelos finalizados
        finalized_models.append({
            'name': model_name,
            'filename': filename,
            'model': final_model
        })
        
    except Exception as e:
        print(f"   Erro ao salvar modelo {model_name}: {str(e)}")
        # Tentar com workaround
        try:
            set_config('fix_imbalance', False)
            final_model = finalize_model(model)
            filename = f"long_parameter_list_{model_name.lower()}_f1_fixed"
            save_model(final_model, filename)
            print(f"   Modelo salvo com workaround como: {filename}.pkl")
        except Exception as e2:
            print(f"   Falha ao salvar mesmo com workaround: {str(e2)}")

print(f"\n✅ {len(finalized_models)} modelos salvos com sucesso!")
print("\nModelos disponíveis para Long Parameter List:")
for model_info in finalized_models:
    print(f"  - {model_info['name']}: {model_info['filename']}.pkl")

Salvando os 5 melhores modelos para Long Parameter List detection...

1. Processando modelo: LGBMClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: lgbmclassifier_long_parameter_list.pkl

2. Processando modelo: KNeighborsClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: kneighborsclassifier_long_parameter_list.pkl

3. Processando modelo: LinearDiscriminantAnalysis
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: lineardiscriminantanalysis_long_parameter_list.pkl

4. Processando modelo: QuadraticDiscriminantAnalysis
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: quadraticdiscriminantanalysis_long_parameter_list.pkl

5. Processando modelo: LogisticRegression
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: logisticregression_long_parameter_list.pkl

✅ 5 modelos salvos com sucesso!

Modelos disponíveis para Long Parameter List:
  - LGBMClas